In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(r"C:\\Users\\Edison New\\Desktop\\HWG Q1 Reporting 2025\\Datasetsets shared\\Filtered_training_data.csv")

In [3]:
data.columns

Index(['guid', 'region_name', 'district_name', 'cluster_name', 'village_name',
       'village_id', 'cohort', 'cycle', 'household_id', 'name', 'gender',
       'age', 'contact', 'training_latitude', 'training_longitude',
       'training_day', 'trainer_name', 'trainer_gender',
       'trainer_organization', 'trainer_title', 'trainer_phone_number',
       'number_of_village_households', 'number_of_households',
       'number_of_male_households', 'number_of_female_households',
       'number_of_male_youth_households', 'number_of_female_youth_households',
       'performance_rating', 'hours_to_complete', 'training_challenge',
       'general_comment', 'number_of_males_to_vote_equality_before',
       'number_of_females_to_vote_equality_before',
       'number_of_males_to_vote_equality_after',
       'number_of_females_to_vote_equality_after',
       'number_of_males_understand_importance_of_iga_before',
       'number_of_females_understand_importance_of_iga_before',
       'number_of_male

In [4]:
data1 = data.pivot_table(
    index=['district_name', 'cohort', 'cycle', 'type'],
    values='household_id',
    aggfunc='count'
).rename(columns={'household_id': 'No_of_HH_attended_training'})
data1= data1.reset_index()

In [12]:

data1['Total_population']=np.where(data1['district_name']=="Kitagwenda",2014,
                                                  np.where(data1['district_name']=="Rakai",2087,np.nan))

In [11]:
data1

,district_name,cohort,cycle,type,No_of_HH_attended_training,Total_population
0,Kitagwenda,2025,A,CATs / Agriculture Champions,126,2031.0
1,Kitagwenda,2025,A,Coffee Champions - ToT 1,22,2031.0
2,Kitagwenda,2025,A,HHT 1 - Day 2 - VSLAs,1940,2031.0
3,Kitagwenda,2025,A,HHT 1 - Mindset and behavioral change,1933,2031.0
4,Kitagwenda,2025,A,HOR 1,1939,2031.0
5,Kitagwenda,2025,A,Village Champions,26,2031.0
6,Kitagwenda,2025,A,WASH - Training 1,1937,2031.0
7,Kitagwenda,2025,A,WASH Champions Training,98,2031.0
8,Rakai,2025,A,CATs / Agriculture Champions,109,2087.0
9,Rakai,2025,A,Coffee Champions - ToT 1,177,2087.0


In [13]:
champions_designed=pd.read_csv("C:\\Users\\Edison New\\Desktop\\HWG Q1 Reporting 2025\\Datasetsets shared\\designed champions_clean.csv")

In [18]:
champions_designed=champions_designed.groupby(['district_name','champion_type'])['Total_champions'].sum().reset_index()
champions_designed

,district_name,champion_type,Total_champions
0,Kitagwenda,agric champion,103
1,Kitagwenda,coffee champion,103
2,Kitagwenda,village champion,26
3,Kitagwenda,wash champion,103
4,Rakai,agric champion,104
5,Rakai,coffee champion,104
6,Rakai,village champion,17
7,Rakai,wash champion,104


In [ ]:
k=['WASH - Training 1','HHT 1 - Mindset and behavioral change','HHT 1 - Day 2 - VSLAs','HOR 1']
if k in data1['type'].unique():
    data1['percentage_attendance']=data1['No_of_HH_attended_training']/data1['Total_population']
else:
    data1[]

In [ ]:
data1['percentage_attendance']=data1['No_of_HH_attended_training']/data1['Total_population']

KeyError: 'No_HHs_attended_agric_training'